In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Se buscará primero determinar cómo el acceso a Internet a evolucionado a lo largo de los años y cómo el acceso al servicio de Televisión se ha visto afectado, partiendo de la premisa de que desde la llegada de Internet y los servicios de streaming, la Televisión ha decaído y cada vez menos personas la van a adquirir como parte de sus paquetes.
Se cargarán los datos de internet como son:
- Las velocidades
- Los accesos por tecnologías
- Mbps medios de bajada por provincia
- La penetración de internet por población y hogares
- Ingresos totales

Una vez obtenida y limpiada esta información para el internet, se procederá a revisar la Televisión, la cual se encuentra en el archivo `television_EDA.ipynb`, en particular:
- Los clientes de TV por provincia 
- La penetración de TV por provincia
- Los ingresos totales
- Las tecnologías usadas para el servicio de TV

In [7]:
internet_velocidad = pd.read_excel('data/Internet.xlsx', sheet_name='Velocidad_sin_Rangos')
internet_tecnologias = pd.read_excel('data/Internet.xlsx', sheet_name='Accesos Por Tecnología')
internet_mbps = pd.read_excel('data/Internet.xlsx', sheet_name='Velocidad % por prov')
internet_penetracion = pd.read_excel('data/Internet.xlsx', sheet_name='Penetracion-totales')
internet_ingresos = pd.read_excel('data/Internet.xlsx', sheet_name='Ingresos ')

Para facilitar el análisis, se comenzará por hacer un drop de todas las filas que contentan algún dato nulo o que no aporte información relevante. Además, a fin de simplificar los datos, se considerarán sólo las tendencias anuales, por tanto los trimestres no se tendrán en cuenta para estos análisis.

In [19]:
internet_velocidad.dropna(inplace=True)
internet_tecnologias.dropna(inplace=True)
internet_mbps.dropna(inplace=True)
internet_penetracion.dropna(inplace=True)
internet_ingresos.dropna(inplace=True)

## 1. Limpieza y agrupamientos
Se comienza realizando una limpieza de datos y agrupando los datos, ya que los archivos están en un formato wide, por lo que las agrupaciones ayudarán más adelante cuando se visualicen los datos y se haga el dashboard.

In [20]:
internet_velocidad.describe()

,Año,Trimestre,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,...,2048 Mbps,2529 Mbps,3072 Mbps,4000 Mbps,5000 Mbps,5125 Mbps,6000 Mbps,7000 Mbps,8000 Mbps,10000 Mbps
count,624.000000,624.000000,624.000000,624.000000,624.000000,624.000000,624.000000,624.000000,624.000000,624.000000,...,624.000000,624.000000,624.000000,624.000000,624.000000,624.000000,624.000000,624.000000,624.000000,624.000000
mean,2020.500000,2.500000,9905.139423,3.469551,3.966346,1.956731,2236.834936,40.467949,1150.025641,4706.605769,...,6.903846,0.032051,1.818910,0.113782,0.059295,0.006410,0.054487,0.158654,0.012821,0.073718
std,1.907996,1.152846,17973.450593,10.749551,29.632806,14.618851,15902.906210,69.387753,4676.714406,7344.002637,...,76.733746,0.800641,20.254423,1.209195,0.719825,0.079871,0.605151,1.441916,0.159742,0.701533
min,2017.000000,1.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2019.000000,1.000000,589.500000,0.000000,0.000000,0.000000,20.750000,0.000000,0.000000,690.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2020.500000,2.500000,4467.500000,0.000000,0.000000,0.000000,47.000000,16.000000,4.000000,2003.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2022.000000,4.000000,13300.750000,1.000000,0.000000,0.000000,82.250000,60.000000,318.000000,5196.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2024.000000,4.000000,125147.000000,102.000000,225.000000,111.000000,238902.000000,609.000000,35185.000000,49473.000000,...,860.000000,20.000000,227.000000,15.000000,9.000000,1.000000,8.000000,18.000000,2.000000,9.000000


In [22]:
internet_velocidad

,Año,Trimestre,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",...,2048 Mbps,2529 Mbps,3072 Mbps,4000 Mbps,5000 Mbps,5125 Mbps,6000 Mbps,7000 Mbps,8000 Mbps,10000 Mbps
0,2024.0,1.0,Buenos Aires,125147.0,18.0,0.0,0.0,25815.0,169.0,4611.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024.0,1.0,Capital Federal,0.0,31.0,0.0,0.0,451.0,34.0,1307.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024.0,1.0,Catamarca,70.0,0.0,0.0,0.0,34.0,37.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024.0,1.0,Chaco,3816.0,0.0,0.0,0.0,234.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024.0,1.0,Chubut,16640.0,8.0,0.0,0.0,69.0,34.0,836.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,2017.0,4.0,Santa Cruz,0.0,0.0,0.0,0.0,68.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
620,2017.0,4.0,Santa Fe,14119.0,0.0,0.0,0.0,3109.0,0.0,7637.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
621,2017.0,4.0,Santiago Del Estero,0.0,0.0,0.0,0.0,41.0,0.0,909.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
622,2017.0,4.0,Tierra Del Fuego,0.0,0.0,0.0,0.0,6.0,0.0,282.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# Agrupar por año y provincia para obtener el total de accesos a cada velocidad
velocidad_sin_rangos_summary = internet_velocidad.drop(columns=['Trimestre']).groupby(['Año', 'Provincia']).sum().reset_index()
velocidad_sin_rangos_summary

,Año,Provincia,Otros,"0,256 Mbps","0,375 Mbps","0,625 Mbps","0,5 Mbps","0,512 Mbps","0,75 Mbps",1 Mbps,...,2048 Mbps,2529 Mbps,3072 Mbps,4000 Mbps,5000 Mbps,5125 Mbps,6000 Mbps,7000 Mbps,8000 Mbps,10000 Mbps
0,2017.0,Buenos Aires,114182.0,0.0,0.0,0.0,3865.0,0.0,35185.0,19861.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2017.0,Capital Federal,0.0,0.0,0.0,0.0,314.0,0.0,701.0,30710.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2017.0,Catamarca,2151.0,0.0,0.0,0.0,34.0,0.0,1.0,3181.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2017.0,Chaco,1035.0,0.0,0.0,0.0,70.0,0.0,1.0,3431.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2017.0,Chubut,0.0,0.0,0.0,0.0,67.0,0.0,3086.0,1227.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,2024.0,Santa Cruz,33803.0,4.0,0.0,0.0,26.0,20.0,218.0,209.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
188,2024.0,Santa Fe,16168.0,32.0,0.0,0.0,335.0,101.0,5.0,6412.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,9.0
189,2024.0,Santiago Del Estero,917.0,2.0,0.0,0.0,39.0,139.0,908.0,226.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
190,2024.0,Tierra Del Fuego,7121.0,0.0,0.0,0.0,11.0,11.0,318.0,516.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
internet_tecnologias

,Año,Trimestre,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2024,1,Buenos Aires,203297.0,2790663.0,1799013.0,150290.0,101294.0,5044557.0
1,2024,1,Capital Federal,70108.0,1177475.0,203756.0,4493.0,48879.0,1504711.0
2,2024,1,Catamarca,5901.0,11025.0,56542.0,1384.0,81.0,74933.0
3,2024,1,Chaco,11343.0,58929.0,64463.0,8404.0,12404.0,155543.0
4,2024,1,Chubut,26306.0,77217.0,26356.0,30867.0,9930.0,170676.0
...,...,...,...,...,...,...,...,...,...
979,2014,1,Santa Cruz,18927.0,100.0,381.0,4092.0,3264.0,26764.0
980,2014,1,Santa Fe,322662.0,174296.0,3059.0,5951.0,644.0,506612.0
981,2014,1,Santiago Del Estero,32567.0,3598.0,19.0,915.0,34.0,37133.0
982,2014,1,Tierra Del Fuego,21618.0,2837.0,648.0,1.0,2934.0,28038.0


In [29]:
# Accesos Por Tecnología: Se limpian los datos y agrupan por año y provincia
accesos_tecnologia_df_clean = internet_tecnologias.drop(columns=[col for col in internet_tecnologias.columns if internet_tecnologias[col].isnull().mean() > 0.8])
accesos_tecnologia_summary = accesos_tecnologia_df_clean[['Año','Provincia',	'ADSL',	'Cablemodem',	'Fibra óptica',	'Wireless',	'Otros',	'Total']].groupby(['Año', 'Provincia']).sum().reset_index()
accesos_tecnologia_summary

,Año,Provincia,ADSL,Cablemodem,Fibra óptica,Wireless,Otros,Total
0,2014,Buenos Aires,6275077.0,4207132.0,485970.0,71325.0,130890.0,11170394.0
1,2014,Capital Federal,1799146.0,3322198.0,70537.0,7706.0,58891.0,5258478.0
2,2014,Catamarca,98078.0,774.0,65.0,0.0,177.0,99094.0
3,2014,Chaco,214637.0,40621.0,120.0,1.0,237.0,255616.0
4,2014,Chubut,199894.0,2553.0,3505.0,45845.0,26622.0,278419.0
...,...,...,...,...,...,...,...,...
283,2019 *,Santa Cruz,31987.0,31928.0,1730.0,19572.0,8208.0,93425.0
284,2019 *,Santa Fe,984854.0,1072655.0,52547.0,69464.0,61976.0,2241496.0
285,2019 *,Santiago Del Estero,74286.0,197528.0,461.0,8365.0,69099.0,349739.0
286,2019 *,Tierra Del Fuego,59369.0,75735.0,2925.0,0.0,4459.0,142488.0


In [26]:
internet_mbps

,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,2024,1,Buenos Aires,159.130000
1,2024,1,Capital Federal,229.940000
2,2024,1,Catamarca,91.450000
3,2024,1,Chaco,107.510000
4,2024,1,Chubut,20.500000
...,...,...,...,...
979,2014,1,Santa Cruz,3.315991
980,2014,1,Santa Fe,3.147407
981,2014,1,Santiago Del Estero,2.949472
982,2014,1,Tierra Del Fuego,3.446888


In [28]:
# Velocidad % por prov: Se agrupan por año y provincia, se ignoran los trimestres para facilitar la interpretación a largo plazo de las velocidades
velocidad_por_prov_df_clean = internet_mbps.dropna()
velocidad_por_prov_summary = velocidad_por_prov_df_clean[['Año','Provincia','Mbps (Media de bajada)']].groupby(['Año', 'Provincia']).mean().reset_index()
velocidad_por_prov_summary

,Año,Provincia,Mbps (Media de bajada)
0,2014,Buenos Aires,3.938247
1,2014,Capital Federal,4.373310
2,2014,Catamarca,2.992319
3,2014,Chaco,3.528954
4,2014,Chubut,3.168178
...,...,...,...
259,2024,Santa Cruz,31.600000
260,2024,Santa Fe,100.200000
261,2024,Santiago Del Estero,44.900000
262,2024,Tierra Del Fuego,21.540000


In [33]:
internet_penetracion

,Año,Trimestre,Accesos por cada 100 hogares,Accesos por cada 100 hab,Periodo
0,2024,1,78.890000,24.790000,Ene-Mar 2024
1,2023,4,78.560000,24.660000,Oct-Dic 2023
2,2023,3,77.840000,24.410000,Jul-Sept 2023
3,2023,2,77.020000,24.140000,Abr-Jun 2023
4,2023,1,77.200000,24.170000,Ene-Mar 2023
5,2022,4,77.210000,24.150000,Oct-Dic 2022
6,2022,3,76.640000,23.950000,Jul-Sept 2022
7,2022,2,75.965424,23.720442,Abr-Jun 2022
8,2022,1,73.878803,23.049171,Ene-Mar 2022
9,2021,4,73.176397,22.810702,Oct-Dic 2021


In [42]:
# Penetracion-totales: Se limpian y agrupar por año
penetracion_totales_df_clean = internet_penetracion.drop(columns=['Trimestre','Periodo'])
penetracion_totales_summary = penetracion_totales_df_clean.groupby(['Año']).mean().reset_index()
penetracion_totales_summary

,Año,Accesos por cada 100 hogares,Accesos por cada 100 hab
0,2014,50.144852,15.259364
1,2015,52.053712,15.916863
2,2016,53.214464,16.335457
3,2017,55.936220,17.207378
4,2018,60.021969,18.515000
5,2019,63.929266,19.779117
6,2020,65.249676,20.250465
7,2021,70.235327,21.867256
8,2022,75.923557,23.717403
9,2023,77.655000,24.345000


In [40]:
internet_ingresos

,Año,Trimestre,Ingresos (miles de pesos),Periodo
0,2024,1,2.804191e+08,Ene-Mar 2024
1,2023,4,1.673760e+08,Oct-Dic 2023
2,2023,3,1.331066e+08,Jul-Sept 2023
3,2023,2,1.180603e+08,Jun-Mar 2023
4,2033,1,1.041350e+08,Ene-Mar 2023
5,2022,4,7.244348e+07,Oct-Dic 2022
6,2022,3,6.408678e+07,Jul-Sept 2022
7,2022,2,6.041950e+07,Abr-Jun 2022
8,2022,1,5.521931e+07,Ene-Mar 2022
9,2021,4,4.546789e+07,Oct-Dic 2021


In [41]:
# Ingresos: Se calcula la suma anual de ingresos por concepto de Internet
ingresos_df_clean = internet_ingresos.drop(columns=['Trimestre','Periodo'])
ingresos_summary = ingresos_df_clean.groupby(['Año']).sum().reset_index()
ingresos_summary

,Año,Ingresos (miles de pesos)
0,2014,1.368395e+07
1,2015,2.010881e+07
2,2016,2.686751e+07
3,2017,3.967197e+07
4,2018,5.953959e+07
5,2019,8.677253e+07
6,2020,1.275858e+08
7,2021,1.633839e+08
8,2022,2.521691e+08
9,2023,4.185429e+08
